In [56]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'c:\\Users\\Joaquín Amat\\Documents\\GitHub\\skforecast'

In [57]:
from skforecast.model_selection.model_selection_new import TimeSeriesFold, TimeSeriesOneStepAhead
from skforecast.model_selection.model_selection import _create_backtesting_folds
import pandas as pd
import numpy as np
import itertools

In [58]:
n=100
my_series = pd.Series(range(n), index=pd.date_range(start='2020-01-01', periods=n, freq='D'))

cv = TimeSeriesFold(
        window_size=5,
        initial_train_size=50,
        steps=10,
        refit=False,
        fixed_train_size=True,
        gap=0,
        skip_folds=None,
        allow_incomplete_fold=True,
        return_all_indexes=False,
        differentiation=None,
        verbose=True
    )

folds = cv.split(my_series)
folds

Information of folds
--------------------
Number of observations used for initial training: 50
Number of observations used for backtesting: 50
    Number of folds: 5
    Number skipped folds: 0 
    Number of steps per fold: 10
    Number of steps to exclude from the end of each train set before test (gap): 0

Fold: 0
    Training:   2020-01-01 00:00:00 -- 2020-02-19 00:00:00  (n=50)
    Validation: 2020-02-20 00:00:00 -- 2020-02-29 00:00:00  (n=10)
Fold: 1
    Training:   No training in this fold
    Validation: 2020-03-01 00:00:00 -- 2020-03-10 00:00:00  (n=10)
Fold: 2
    Training:   No training in this fold
    Validation: 2020-03-11 00:00:00 -- 2020-03-20 00:00:00  (n=10)
Fold: 3
    Training:   No training in this fold
    Validation: 2020-03-21 00:00:00 -- 2020-03-30 00:00:00  (n=10)
Fold: 4
    Training:   No training in this fold
    Validation: 2020-03-31 00:00:00 -- 2020-04-09 00:00:00  (n=10)



[[[0, 50], [45, 50], [50, 60], [50, 60], True],
 [[0, 50], [55, 60], [60, 70], [60, 70], False],
 [[0, 50], [65, 70], [70, 80], [70, 80], False],
 [[0, 50], [75, 80], [80, 90], [80, 90], False],
 [[0, 50], [85, 90], [90, 100], [90, 100], False]]

In [59]:
folds = cv.split(my_series, as_data_frame=True)
folds

Information of folds
--------------------
Number of observations used for initial training: 50
    First 0 observation/s in training sets are used for differentiation
Number of observations used for backtesting: 50
    Number of folds: 5
    Number skipped folds: 0 
    Number of steps per fold: 10
    Number of steps to exclude from the end of each train set before test (gap): 0

Fold: 0
    Training:   2020-01-01 00:00:00 -- 2020-02-19 00:00:00  (n=50)
    Validation: 2020-02-20 00:00:00 -- 2020-02-29 00:00:00  (n=10)
Fold: 1
    Training:   No training in this fold
    Validation: 2020-03-01 00:00:00 -- 2020-03-10 00:00:00  (n=10)
Fold: 2
    Training:   No training in this fold
    Validation: 2020-03-11 00:00:00 -- 2020-03-20 00:00:00  (n=10)
Fold: 3
    Training:   No training in this fold
    Validation: 2020-03-21 00:00:00 -- 2020-03-30 00:00:00  (n=10)
Fold: 4
    Training:   No training in this fold
    Validation: 2020-03-31 00:00:00 -- 2020-04-09 00:00:00  (n=10)



,fold,train_start,train_end,last_window_start,last_window_end,test_start,test_end,test_start_with_gap,test_end_with_gap,fit_forecaster
0,0,0,50,45,50,50,60,50,60,True
1,1,0,50,55,60,60,70,60,70,False
2,2,0,50,65,70,70,80,70,80,False
3,3,0,50,75,80,80,90,80,90,False
4,4,0,50,85,90,90,100,90,100,False


In [60]:
cv = TimeSeriesFold(
    window_size=None,
    initial_train_size=50,
    steps=10,
    refit=False,
    fixed_train_size=True,
    gap=0,
    skip_folds=None,
    allow_incomplete_fold=True,
    return_all_indexes=False,
    differentiation=2,
    verbose=True
)

folds = cv.split(my_series)
folds

Information of folds
--------------------
Number of observations used for initial training: 48
    First 2 observation/s in training sets are used for differentiation
Number of observations used for backtesting: 50
    Number of folds: 5
    Number skipped folds: 0 
    Number of steps per fold: 10
    Number of steps to exclude from the end of each train set before test (gap): 0

Fold: 0
    Training:   2020-01-03 00:00:00 -- 2020-02-19 00:00:00  (n=48)
    Validation: 2020-02-20 00:00:00 -- 2020-02-29 00:00:00  (n=10)
Fold: 1
    Training:   No training in this fold
    Validation: 2020-03-01 00:00:00 -- 2020-03-10 00:00:00  (n=10)
Fold: 2
    Training:   No training in this fold
    Validation: 2020-03-11 00:00:00 -- 2020-03-20 00:00:00  (n=10)
Fold: 3
    Training:   No training in this fold
    Validation: 2020-03-21 00:00:00 -- 2020-03-30 00:00:00  (n=10)
Fold: 4
    Training:   No training in this fold
    Validation: 2020-03-31 00:00:00 -- 2020-04-09 00:00:00  (n=10)



[[[0, 50], [], [50, 60], [50, 60], True],
 [[0, 50], [], [60, 70], [60, 70], False],
 [[0, 50], [], [70, 80], [70, 80], False],
 [[0, 50], [], [80, 90], [80, 90], False],
 [[0, 50], [], [90, 100], [90, 100], False]]

In [61]:
cv = TimeSeriesFold(
    window_size=5,
    initial_train_size=50,
    steps=10,
    refit=False,
    fixed_train_size=True,
    gap=0,
    skip_folds=None,
    allow_incomplete_fold=True,
    return_all_indexes=True,
    differentiation=None,
    verbose=True
)

folds = cv.split(my_series)
folds

Information of folds
--------------------
Number of observations used for initial training: 50
Number of observations used for backtesting: 50
    Number of folds: 5
    Number skipped folds: 0 
    Number of steps per fold: 10
    Number of steps to exclude from the end of each train set before test (gap): 0

Fold: 0
    Training:   2020-01-01 00:00:00 -- 2020-02-19 00:00:00  (n=50)
    Validation: 2020-02-20 00:00:00 -- 2020-02-29 00:00:00  (n=10)
Fold: 1
    Training:   No training in this fold
    Validation: 2020-03-01 00:00:00 -- 2020-03-10 00:00:00  (n=10)
Fold: 2
    Training:   No training in this fold
    Validation: 2020-03-11 00:00:00 -- 2020-03-20 00:00:00  (n=10)
Fold: 3
    Training:   No training in this fold
    Validation: 2020-03-21 00:00:00 -- 2020-03-30 00:00:00  (n=10)
Fold: 4
    Training:   No training in this fold
    Validation: 2020-03-31 00:00:00 -- 2020-04-09 00:00:00  (n=10)



[[range(0, 50), range(45, 50), range(50, 60), range(50, 60), True],
 [range(0, 50), range(55, 60), range(60, 70), range(60, 70), False],
 [range(0, 50), range(65, 70), range(70, 80), range(70, 80), False],
 [range(0, 50), range(75, 80), range(80, 90), range(80, 90), False],
 [range(0, 50), range(85, 90), range(90, 100), range(90, 100), False]]

In [62]:
cv = TimeSeriesFold(
    window_size=5,
    initial_train_size=50,
    steps=10,
    refit=True,
    fixed_train_size=True,
    gap=0,
    skip_folds=None,
    allow_incomplete_fold=True,
    return_all_indexes=True,
    differentiation=None,
    verbose=True
)

folds = cv.split(my_series)
folds

Information of folds
--------------------
Number of observations used for initial training: 50
Number of observations used for backtesting: 50
    Number of folds: 5
    Number skipped folds: 0 
    Number of steps per fold: 10
    Number of steps to exclude from the end of each train set before test (gap): 0

Fold: 0
    Training:   2020-01-01 00:00:00 -- 2020-02-19 00:00:00  (n=50)
    Validation: 2020-02-20 00:00:00 -- 2020-02-29 00:00:00  (n=10)
Fold: 1
    Training:   2020-01-11 00:00:00 -- 2020-02-29 00:00:00  (n=50)
    Validation: 2020-03-01 00:00:00 -- 2020-03-10 00:00:00  (n=10)
Fold: 2
    Training:   2020-01-21 00:00:00 -- 2020-03-10 00:00:00  (n=50)
    Validation: 2020-03-11 00:00:00 -- 2020-03-20 00:00:00  (n=10)
Fold: 3
    Training:   2020-01-31 00:00:00 -- 2020-03-20 00:00:00  (n=50)
    Validation: 2020-03-21 00:00:00 -- 2020-03-30 00:00:00  (n=10)
Fold: 4
    Training:   2020-02-10 00:00:00 -- 2020-03-30 00:00:00  (n=50)
    Validation: 2020-03-31 00:00:00 -- 2020-0

[[range(0, 50), range(45, 50), range(50, 60), range(50, 60), True],
 [range(10, 60), range(55, 60), range(60, 70), range(60, 70), True],
 [range(20, 70), range(65, 70), range(70, 80), range(70, 80), True],
 [range(30, 80), range(75, 80), range(80, 90), range(80, 90), True],
 [range(40, 90), range(85, 90), range(90, 100), range(90, 100), True]]

In [63]:
cv = TimeSeriesFold(
    window_size=5,
    initial_train_size=50,
    steps=10,
    refit=True,
    fixed_train_size=False,
    gap=0,
    skip_folds=None,
    allow_incomplete_fold=True,
    return_all_indexes=True,
    differentiation=None,
    verbose=True
)

folds = cv.split(my_series)
folds

Information of folds
--------------------
Number of observations used for initial training: 50
Number of observations used for backtesting: 50
    Number of folds: 5
    Number skipped folds: 0 
    Number of steps per fold: 10
    Number of steps to exclude from the end of each train set before test (gap): 0

Fold: 0
    Training:   2020-01-01 00:00:00 -- 2020-02-19 00:00:00  (n=50)
    Validation: 2020-02-20 00:00:00 -- 2020-02-29 00:00:00  (n=10)
Fold: 1
    Training:   2020-01-01 00:00:00 -- 2020-02-29 00:00:00  (n=60)
    Validation: 2020-03-01 00:00:00 -- 2020-03-10 00:00:00  (n=10)
Fold: 2
    Training:   2020-01-01 00:00:00 -- 2020-03-10 00:00:00  (n=70)
    Validation: 2020-03-11 00:00:00 -- 2020-03-20 00:00:00  (n=10)
Fold: 3
    Training:   2020-01-01 00:00:00 -- 2020-03-20 00:00:00  (n=80)
    Validation: 2020-03-21 00:00:00 -- 2020-03-30 00:00:00  (n=10)
Fold: 4
    Training:   2020-01-01 00:00:00 -- 2020-03-30 00:00:00  (n=90)
    Validation: 2020-03-31 00:00:00 -- 2020-0

[[range(0, 50), range(45, 50), range(50, 60), range(50, 60), True],
 [range(0, 60), range(55, 60), range(60, 70), range(60, 70), True],
 [range(0, 70), range(65, 70), range(70, 80), range(70, 80), True],
 [range(0, 80), range(75, 80), range(80, 90), range(80, 90), True],
 [range(0, 90), range(85, 90), range(90, 100), range(90, 100), True]]

In [64]:
cv = TimeSeriesFold(
    window_size=5,
    initial_train_size=50,
    steps=10,
    refit=True,
    fixed_train_size=False,
    gap=0,
    skip_folds=None,
    allow_incomplete_fold=True,
    return_all_indexes=True,
    differentiation=None,
    verbose=True
)

print(cv)

cv.set_params({'window_size': 10, 'differentiation': 2})
print(cv)

TimeSeriesFold(
    steps=10,
    initial_train_size=50,
    window_size=5,
    differentiation=None,
    refit=True,
    fixed_train_size=False,
    gap=0,
    skip_folds=None,
    allow_incomplete_fold=True,
    return_all_indexes=True,
    verbose=True
)
TimeSeriesFold(
    steps=10,
    initial_train_size=50,
    window_size=10,
    differentiation=2,
    refit=True,
    fixed_train_size=False,
    gap=0,
    skip_folds=None,
    allow_incomplete_fold=True,
    return_all_indexes=True,
    verbose=True
)


## Backtesting

In [65]:
import pandas as pd
import matplotlib.pyplot as plt
from skforecast.datasets import fetch_dataset
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection.model_selection import backtesting_forecaster
from skforecast.model_selection.model_selection import grid_search_forecaster
from skforecast.model_selection.model_selection_new import backtesting_forecaster as backtesting_forecaster_new
from skforecast.model_selection.model_selection_new import grid_search_forecaster as grid_search_forecaster_new
from sklearn.linear_model import Ridge
from sklearn.model_selection import ParameterGrid
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error

In [66]:
# Download data
# ==============================================================================
data = fetch_dataset(
    name="h2o", raw=True, kwargs_read_csv={"names": ["y", "datetime"], "header": 0}
)

# Data preprocessing
# ==============================================================================
data['datetime'] = pd.to_datetime(data['datetime'], format='%Y-%m-%d')
data = data.set_index('datetime')
data = data.asfreq('MS')
data = data[['y']]
data = data.sort_index()

# Train-validation dates
# ==============================================================================
end_train = '2002-01-01 23:59:00'

print(
    f"Train dates      : {data.index.min()} --- {data.loc[:end_train].index.max()}"
    f"  (n={len(data.loc[:end_train])})"
)
print(
    f"Validation dates : {data.loc[end_train:].index.min()} --- {data.index.max()}"
    f"  (n={len(data.loc[end_train:])})"
)

h2o
---
Monthly expenditure ($AUD) on corticosteroid drugs that the Australian health
system had between 1991 and 2008.
Hyndman R (2023). fpp3: Data for Forecasting: Principles and Practice(3rd
Edition). http://pkg.robjhyndman.com/fpp3package/,https://github.com/robjhyndman
/fpp3package, http://OTexts.com/fpp3.
Shape of the dataset: (204, 2)
Train dates      : 1991-07-01 00:00:00 --- 2002-01-01 00:00:00  (n=127)
Validation dates : 2002-02-01 00:00:00 --- 2008-06-01 00:00:00  (n=77)


In [67]:
# Backtesting forecaster
# ==============================================================================
forecaster = ForecasterAutoreg(
                 regressor = LGBMRegressor(random_state=123, verbose=-1),
                 lags      = 15 
             )

cv = TimeSeriesFold(
        initial_train_size=len(data.loc[:end_train]),
        steps=10,
        fixed_train_size=False,
        allow_incomplete_fold=True,
        refit=True,
        gap=0,
    )

metric, predictions = backtesting_forecaster_new(
                          forecaster            = forecaster,
                          y                     = data['y'],
                          cv                    = cv,
                          metric                = 'mean_squared_error',
                          n_jobs                = 'auto',
                          verbose               = True,
                          show_progress         = True  
                      )
metric

Information of folds
--------------------
Number of observations used for initial training: 127
Number of observations used for backtesting: 77
    Number of folds: 8
    Number skipped folds: 0 
    Number of steps per fold: 10
    Number of steps to exclude from the end of each train set before test (gap): 0
    Last fold only includes 7 observations.

Fold: 0
    Training:   1991-07-01 00:00:00 -- 2002-01-01 00:00:00  (n=127)
    Validation: 2002-02-01 00:00:00 -- 2002-11-01 00:00:00  (n=10)
Fold: 1
    Training:   1991-07-01 00:00:00 -- 2002-11-01 00:00:00  (n=137)
    Validation: 2002-12-01 00:00:00 -- 2003-09-01 00:00:00  (n=10)
Fold: 2
    Training:   1991-07-01 00:00:00 -- 2003-09-01 00:00:00  (n=147)
    Validation: 2003-10-01 00:00:00 -- 2004-07-01 00:00:00  (n=10)
Fold: 3
    Training:   1991-07-01 00:00:00 -- 2004-07-01 00:00:00  (n=157)
    Validation: 2004-08-01 00:00:00 -- 2005-05-01 00:00:00  (n=10)
Fold: 4
    Training:   1991-07-01 00:00:00 -- 2005-05-01 00:00:00  (n=

  0%|          | 0/8 [00:00<?, ?it/s]

,mean_squared_error
0,0.00829


In [68]:
cv.split(data['y'])

Information of folds
--------------------
Number of observations used for initial training: 127
Number of observations used for backtesting: 77
    Number of folds: 8
    Number skipped folds: 0 
    Number of steps per fold: 10
    Number of steps to exclude from the end of each train set before test (gap): 0
    Last fold only includes 7 observations.

Fold: 0
    Training:   1991-07-01 00:00:00 -- 2002-01-01 00:00:00  (n=127)
    Validation: 2002-02-01 00:00:00 -- 2002-11-01 00:00:00  (n=10)
Fold: 1
    Training:   1991-07-01 00:00:00 -- 2002-11-01 00:00:00  (n=137)
    Validation: 2002-12-01 00:00:00 -- 2003-09-01 00:00:00  (n=10)
Fold: 2
    Training:   1991-07-01 00:00:00 -- 2003-09-01 00:00:00  (n=147)
    Validation: 2003-10-01 00:00:00 -- 2004-07-01 00:00:00  (n=10)
Fold: 3
    Training:   1991-07-01 00:00:00 -- 2004-07-01 00:00:00  (n=157)
    Validation: 2004-08-01 00:00:00 -- 2005-05-01 00:00:00  (n=10)
Fold: 4
    Training:   1991-07-01 00:00:00 -- 2005-05-01 00:00:00  (n=

[[[0, 127], [], [127, 137], [127, 137], True],
 [[0, 137], [], [137, 147], [137, 147], True],
 [[0, 147], [], [147, 157], [147, 157], True],
 [[0, 157], [], [157, 167], [157, 167], True],
 [[0, 167], [], [167, 177], [167, 177], True],
 [[0, 177], [], [177, 187], [177, 187], True],
 [[0, 187], [], [187, 197], [187, 197], True],
 [[0, 197], [], [197, 204], [197, 204], True]]

In [69]:
# Test equivalence with old version of backtesting_forecaster
# ==============================================================================

## check_inpunt_backtesting must be commented in model_selection.py!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

params = {
    'initial_train_size': [127],
    'steps': [10],
    'fixed_train_size': [True, False],
    'allow_incomplete_fold': [True, False],
    'refit': [True, False, 3],
    'gap': [0, 1, 3],
    'differentiation': [None, 1, 2],
}

params_grid = list(ParameterGrid(params))
forecaster = ForecasterAutoreg(
                 regressor = LGBMRegressor(random_state=123, verbose=-1),
                 lags      = 15 
             )


for params in params_grid:
    print(params)

    ## check_inpunt_backtesting must be commented in model_selection.py!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    metric, predictions = backtesting_forecaster(
                                forecaster            = forecaster,
                                y                     = data['y'],
                                metric                = 'mean_squared_error',
                                n_jobs                = 'auto',
                                verbose               = False,
                                show_progress         = False,
                                # all paramns except 'differentiation' are passed as **params
                                initial_train_size    = params['initial_train_size'],
                                steps                 = params['steps'],
                                fixed_train_size      = params['fixed_train_size'],
                                allow_incomplete_fold = params['allow_incomplete_fold'],
                                refit                 = params['refit'],
                                gap                   = params['gap']
                            )

    cv = TimeSeriesFold(**params)
    metric_new, predictions_new = backtesting_forecaster_new(
                                    forecaster            = forecaster,
                                    y                     = data['y'],
                                    cv                    = cv,
                                    metric                = 'mean_squared_error',
                                    n_jobs                = 'auto',
                                    verbose               = False,
                                    show_progress         = False  
                                )
    
    pd.testing.assert_frame_equal(predictions, predictions_new)
    pd.testing.assert_frame_equal(metric, metric_new)


{'allow_incomplete_fold': True, 'differentiation': None, 'fixed_train_size': True, 'gap': 0, 'initial_train_size': 127, 'refit': True, 'steps': 10}


TypeError: check_backtesting_input() got an unexpected keyword argument 'steps'

In [54]:
# Backtesting with intermittent refit
# ==============================================================================
forecaster = ForecasterAutoreg(
                 regressor = LGBMRegressor(random_state=123, verbose=-1),
                 lags      = 15 
             )

cv = TimeSeriesFold(
    initial_train_size=len(data.loc[:end_train]),
    steps=10,
    fixed_train_size=False,
    allow_incomplete_fold=True,
    refit=3,
    gap=0,
)


metric, predictions = backtesting_forecaster(
                          forecaster            = forecaster,
                          y                     = data['y'],
                          cv                    = cv,
                          metric                = 'mean_squared_error',
                          verbose               = True,
                          show_progress         = True  
                      )
metric

Information of folds
--------------------
Number of observations used for initial training: 127
Number of observations used for backtesting: 77
    Number of folds: 8
    Number skipped folds: 0 
    Number of steps per fold: 10
    Number of steps to exclude from the end of each train set before test (gap): 0
    Last fold only includes 7 observations.

Fold: 0
    Training:   1991-07-01 00:00:00 -- 2002-01-01 00:00:00  (n=127)
    Validation: 2002-02-01 00:00:00 -- 2002-11-01 00:00:00  (n=10)
Fold: 1
    Training:   No training in this fold
    Validation: 2002-12-01 00:00:00 -- 2003-09-01 00:00:00  (n=10)
Fold: 2
    Training:   No training in this fold
    Validation: 2003-10-01 00:00:00 -- 2004-07-01 00:00:00  (n=10)
Fold: 3
    Training:   1991-07-01 00:00:00 -- 2004-07-01 00:00:00  (n=157)
    Validation: 2004-08-01 00:00:00 -- 2005-05-01 00:00:00  (n=10)
Fold: 4
    Training:   No training in this fold
    Validation: 2005-06-01 00:00:00 -- 2006-03-01 00:00:00  (n=10)
Fold: 5
 

  0%|          | 0/8 [00:00<?, ?it/s]

,mean_squared_error
0,0.008878


In [31]:
# Backtesting with intermittent refit
# ==============================================================================
forecaster = ForecasterAutoreg(
                 regressor = LGBMRegressor(random_state=123, verbose=-1),
                 lags      = 15,
                 differentiation=1
             )

cv = TimeSeriesFold(
    initial_train_size=len(data.loc[:end_train]),
    steps=10,
    fixed_train_size=False,
    allow_incomplete_fold=True,
    refit=False,
    gap=0,
)


metric, predictions = backtesting_forecaster(
                          forecaster            = forecaster,
                          y                     = data['y'],
                          cv                    = cv,
                          metric                = 'mean_squared_error',
                          verbose               = True,
                          show_progress         = True  
                      )
metric

TypeError: backtesting_forecaster() got an unexpected keyword argument 'cv'

In [70]:
n=100
my_series = pd.Series(range(n), index=pd.date_range(start='2020-01-01', periods=n, freq='D'))

cv = TimeSeriesOneStepAhead(
        window_size=5,
        initial_train_size=50,
        return_all_indexes=False,
        differentiation=None,
        verbose=True
    )

folds = cv.split(my_series, as_data_frame=True)
folds

,fold,train_start,train_end,test_start,test_end,fit_forecaster
0,0,0,50,50,100,True


In [74]:
cv = TimeSeriesOneStepAhead(
        window_size=5,
        initial_train_size=50,
        return_all_indexes=True,
        differentiation=None,
        verbose=True
    )

folds = cv.split(my_series, as_data_frame=True)
folds

,fold,train_index,test_index,fit_forecaster
0,0,"[(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13...","[(50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, ...",True
